In [1]:
import pandas as pd
import requests
import os

In [2]:
def ajustar_nomes_arquivos():
    diretorio_base = "Dados/Por_Municipio"
    pastas = ["Auxilio_Emergencial", "Auxilio_Brasil", "Bolsa_Familia"]

    for pasta in pastas:
        pasta_caminho = os.path.join(diretorio_base, pasta)

        if os.path.exists(pasta_caminho) and os.path.isdir(pasta_caminho):
            for ano in os.listdir(pasta_caminho):
                ano_caminho = os.path.join(pasta_caminho, ano)

                if os.path.exists(ano_caminho) and os.path.isdir(ano_caminho):
                    for arquivo in os.listdir(ano_caminho):
                        nome_arquivo, extensao_arquivo = os.path.splitext(arquivo)
                        nome_arquivo = nome_arquivo.split('_')[0]
                        nome = pasta.split("_")
                        novo_nome = nome_arquivo + "_" + nome[0] + nome[1] + extensao_arquivo
                        # print(novo_nome)

                        arquivo_caminho_antigo = os.path.join(ano_caminho, arquivo)
                        arquivo_caminho_novo = os.path.join(ano_caminho, novo_nome)

                        os.rename(arquivo_caminho_antigo, arquivo_caminho_novo)
                        print(f"Arquivo renomeado: {arquivo_caminho_novo}")


def obter_dados_por_beneficiario(codigo_ibge, mes_ano, beneficio):
    if beneficio == 'bolsa-familia':
        beneficio+='-disponivel'
    elif beneficio == 'auxilio-brasil':
        beneficio+='-sacado'

    url = f"https://api.portaldatransparencia.gov.br/api-de-dados/{beneficio}-beneficiario-por-municipio"
    headers = {"chave-api-dados": "07e464f446e88bb0c105f9cea69b7f45"}

    page = 1
    dados = []

    while True:
        params = {
            "codigoIbge": codigo_ibge,
            "mesAno": mes_ano,
            "pagina": str(page)
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        data = response.json()
        if not data:
            break
        for item in data:
            if beneficio.split('-')[1] == 'brasil':
                dados.append([
                    item['dataMesReferencia'],
                    item['municipio']['nomeIBGE'],
                    item['beneficiarioAuxilioBrasil']['cpfFormatado'],
                    item['beneficiarioAuxilioBrasil']['nis'],
                    item['beneficiarioAuxilioBrasil']['nome'],
                    item['valorSaque']
                ])
            
            # elif beneficio.split('-')[1] == 'familia':
            else:
                dados.append([
                    item['dataMesReferencia'],
                    item['municipio']['nomeIBGE'],
                    item['titularBolsaFamilia']['cpfFormatado'],
                    item['titularBolsaFamilia']['nis'],
                    item['titularBolsaFamilia']['nome'],
                    item['quantidadeDependentes'],
                    item['valor']
                ])
        page += 1
    return dados

## Download dos dados

### Beneficiario

Opções de beneficio:
- bolsa-familia
- auxilio-brasil
- auxilio-emergencial

In [7]:
# Utilização da função para obter os dados para todos os meses do ano
try:
    codigo_ibge = "3305752"
    beneficio = 'auxilio-brasil'
    anos = [str(ano) for ano in range(2020,2024)] # Adicione todos os anos que deseja incluir
    meses = [str(mes).zfill(2) for mes in range(1, 13)]  # Todos os meses do ano

    if beneficio == 'auxilio-brasil':
        header = ['Mes Referência', 'Município', 'CPF Beneficiário', 'NIS Beneficiário', 'Beneficiário', 'Valor']
    elif beneficio == 'bolsa-familia':
        header = ['Mes Referência', 'Município', 'CPF Beneficiário', 'NIS Beneficiário', 'Beneficiário', 'Quantidade Dependentes', 'Valor']
    elif beneficio == 'bpc':
        header = ['Mes Referência', 'Município', 'CPF Beneficiário', 'NIS Beneficiário', 'Beneficiário', 'Representante Legal', 'Concedido Judicialmente', 'Menor de 16 anos', 'Valor']
    else:
        header = ['Mes Referência','Município','CPF Beneficiário','NIS Beneficiário','Beneficiário','Observação','Enquadramento','Parcela','Valor']

    # Para cada ano
    for ano in anos:
        # De cada mês
        for mes in meses:
            mes_ano = ano + mes
            dados = obter_dados_por_beneficiario(codigo_ibge, mes_ano, beneficio)

            # Se os dados não estiverem vazios
            if dados:
                # Cria a pasta para o ano, caso ela não exista
                pasta_ano = f"Dados/Por_Beneficiario/{beneficio}/{ano}"
                if not os.path.exists(pasta_ano):
                    os.makedirs(pasta_ano)

                # Salva os dados em um arquivo CSV
                nome_arquivo = f"{mes_ano}_{beneficio}.csv"
                caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)
                df = pd.DataFrame(dados, columns=header)
                df.to_csv(caminho_arquivo, index=False)
                print(f"Dados para o mês {mes} do ano {ano} foram salvos.")
            else:
                print(f"Dados para o mês {mes} do ano {ano} estava vazio.")
            

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err}")
except Exception as err:
    print(f"Erro: {err}")

Dados para o mês 01 do ano 2020 estava vazio.
Dados para o mês 02 do ano 2020 estava vazio.
Dados para o mês 03 do ano 2020 estava vazio.
Dados para o mês 04 do ano 2020 estava vazio.
Dados para o mês 05 do ano 2020 estava vazio.
Dados para o mês 06 do ano 2020 estava vazio.
Dados para o mês 07 do ano 2020 estava vazio.
Dados para o mês 08 do ano 2020 estava vazio.
Dados para o mês 09 do ano 2020 estava vazio.
Dados para o mês 10 do ano 2020 estava vazio.
Dados para o mês 11 do ano 2020 estava vazio.
Dados para o mês 12 do ano 2020 estava vazio.
Dados para o mês 01 do ano 2021 estava vazio.
Dados para o mês 02 do ano 2021 estava vazio.
Dados para o mês 03 do ano 2021 estava vazio.
Dados para o mês 04 do ano 2021 estava vazio.
Dados para o mês 05 do ano 2021 estava vazio.
Dados para o mês 06 do ano 2021 estava vazio.
Dados para o mês 07 do ano 2021 estava vazio.
Dados para o mês 08 do ano 2021 estava vazio.
Dados para o mês 09 do ano 2021 estava vazio.
Dados para o mês 10 do ano 2021 es

### Beneficio por municipio

Opções de beneficio:
- bolsa-familia
- auxilio-brasil
- auxilio-emergencial

In [5]:
def obter_dados_por_municipio(codigo_ibge, mes_ano, beneficio):
    url = f"https://api.portaldatransparencia.gov.br/api-de-dados/{beneficio}-por-municipio"
    headers = {"chave-api-dados": "07e464f446e88bb0c105f9cea69b7f45"}

    page = 1
    dados = []

    while True:
        params = {
            "codigoIbge": codigo_ibge,
            "mesAno": mes_ano,
            "pagina": str(page)
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        
        data = response.json()
        if not data:
            break

        for item in data:
            dados.append([
                item['dataReferencia'],
                item['municipio']['nomeIBGE'],
                item['tipo']['descricao'],
                item['tipo']['descricaoDetalhada'],
                item['quantidadeBeneficiados'],
                item['valor']
            ])

        page += 1
    return dados

In [ ]:
header = ['Mes Referência', 'Município', 'Benefício', 'Descrição Benefício', 'Quantidade Beneficiário', 'Valor']

# Utilização da função para obter os dados para todos os meses do ano
try:
    codigo_ibge = "3305752"
    beneficio = 'bpc'
    anos = [str(ano) for ano in range(2016,2024)] # Adicione todos os anos que deseja incluir
    meses = [str(mes).zfill(2) for mes in range(1, 13)]  # Todos os meses do ano

    # Para cada ano
    for ano in anos:
        # De cada mês
        for mes in meses:
            mes_ano = ano + mes
            dados = obter_dados_por_municipio(codigo_ibge, mes_ano, beneficio)

            # Se os dados não estiverem vazios
            if dados:
                # Cria a pasta para o ano, caso ela não exista
                pasta_ano = f"Dados/Por_Municipio/{beneficio}/{ano}"
                if not os.path.exists(pasta_ano):
                    os.makedirs(pasta_ano)

                # Salva os dados em um arquivo CSV
                nome_arquivo = f"{mes_ano}_{beneficio}.csv"
                caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)
                df = pd.DataFrame(dados, columns=header)
                df.to_csv(caminho_arquivo, index=False)
                print(f"Dados para o mês {mes} do ano {ano} foram salvos.")
            else:
                print(f"Dados para o mês {mes} do ano {ano} estava vazio.")
            

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err}")
except Exception as err:
    print(f"Erro: {err}")

In [ ]:
ajustar_nomes_arquivos()